In [1]:
#Run this, and run the output
!wget -qO- https://raw.githubusercontent.com/unslothai/unsloth/main/unsloth/_auto_install.py | python -

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
#Run this in case all else failed
# !pip install -U torch==2.3.1
# !pip install torchvision==0.17.2

In [1]:
#Ideally just need to run this to setup everything
!pip install --upgrade pip && pip install "unsloth[cu121-torch220] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-if2jwnsq/unsloth_a5cc87ccbf1846c0a25670e1e6db51c4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-if2jwnsq/unsloth_a5cc87ccbf1846c0a25670e1e6db51c4
  Resolved https://github.com/unslothai/unsloth.git to commit 8558bc92b06f9128499484ef737fa71b966ffc23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://download.pytorch.org/whl/cu121/xformers-0.0.24-cp310-cp310-manylinux2014_x86_64.whl (218.2 MB)


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

base_model_name = "unsloth/Llama-3.2-3B-Instruct"
finetuning_dataset = "NyanDoggo/bird-cot-reasoning"
split = "train"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset(finetuning_dataset, split = split)
dataset = dataset.map(formatting_prompts_func, batched = True,)
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = -1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA H100 80GB HBM3. Max memory: 79.209 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.2.0+cu121. CUDA: 9.0. CUDA Toolkit: 12.1. Triton: 2.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.24. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth 2024.11.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


README.md:   0%|          | 0.00/142 [00:00<?, ?B/s]

vanilla_data_spider_train-alpaca.json:   0%|          | 0.00/21.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/7000 [00:00<?, ? examples/s]

In [3]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H100 80GB HBM3. Max memory = 79.209 GB.
14.844 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 875
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,1.142400
2,1.113700
3,1.274400
4,1.198000
5,1.063900
6,1.037900
7,0.954200
8,1.202900
9,0.934800
10,0.992400


In [ ]:
#Run this to test the model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Solve the following sql problem: How many heads of the departments are older than 56 ? Schema: SCHEMA: - Table: department - Column: Department_ID - Samples: [1, 2, 3, 4, 5] - Column: Name - Samples: [State, Treasury, Defense, Justice, Interior] - Column: Creation - Samples: [1789, 1789, 1947, 1870, 1849] - Column: Ranking - Samples: [1, 2, 3, 4, 5] - Column: Budget_in_Billions - Samples: [9.96, 11.1, 439.3, 23.4, 10.7] - Column: Num_Employees - Samples: [30266.0, 115897.0, 3000000.0, 112557.0, 71436.0] - Table: head - Column: head_ID - Samples: [1, 2, 3, 4, 5] - Column: name - Samples: [Tiger Woods, Sergio García, K. J. Choi, Dudley Hart, Jeff Maggert] - Column: born_state - Samples: [Alabama, California, Alabama, California, Delaware] - Column: age - Samples: [67.0, 68.0, 69.0, 52.0, 53.0] - Table: management - Column: department_ID - Samples: [2, 15, 2, 7, 11] - Column: head_ID - Samples: [5, 4, 6, 3, 10] - Column: temporary_acting - Samples: [Yes, Yes, Yes, No, No]", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
#Run this to save the model and/or to push to huggingface
if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("<Hugging Face Repo URL here>", tokenizer, save_method = "merged_16bit", token = "<huggingface token here>")